In [1]:
from dotenv import load_dotenv, find_dotenv
from typing import Dict
import json
import os
import requests

In [2]:
def load_emaps_api_key():
    load_dotenv(find_dotenv())
    api_key = os.getenv('ELECTRICITY_MAPS_API_KEY')
    if not api_key:
        print('No API key found in environment')
    else:
        return api_key

## Sample: Latest Carbon intensity in Glasgow, Scotland

In [3]:
coordinates = {
    "lat": 55.8287,
    "lon": -4.2611
}
url= f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
print("Endpoint: " + str(url))

Endpoint: https://api.electricitymap.org/v3/carbon-intensity/latest?lat=55.8287&lon=-4.2611


In [4]:
request = requests.get(url, headers={'auth-token': load_emaps_api_key()})

In [5]:
json.loads(request.content)

{'zone': 'GB',
 'carbonIntensity': 234,
 'datetime': '2024-11-10T22:00:00.000Z',
 'updatedAt': '2024-11-10T21:43:10.048Z',
 'createdAt': '2024-11-07T22:45:16.884Z',
 'emissionFactorType': 'lifecycle',
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

## Live power breakdown within Glasgow, Scotland

In [6]:
url = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
print("Endpoint: " + str(url))

Endpoint: https://api.electricitymap.org/v3/power-breakdown/latest?lat=55.8287&lon=-4.2611


In [7]:
request = requests.get(url, headers={'auth-token': load_emaps_api_key()})
power_breakdown = json.loads(request.content)

In [8]:
power_breakdown

{'zone': 'GB',
 'datetime': '2024-11-10T22:00:00.000Z',
 'updatedAt': '2024-11-10T21:43:10.048Z',
 'createdAt': '2024-11-07T22:45:16.884Z',
 'powerConsumptionBreakdown': {'nuclear': 5364,
  'geothermal': 0,
  'biomass': 3034,
  'coal': 0,
  'wind': 8795,
  'solar': 0,
  'hydro': 1172,
  'gas': 13258,
  'oil': 2,
  'unknown': 181,
  'hydro discharge': 66,
  'battery discharge': 0},
 'powerProductionBreakdown': {'nuclear': 4779,
  'geothermal': None,
  'biomass': 3038,
  'coal': 0,
  'wind': 8794,
  'solar': 0,
  'hydro': 127,
  'gas': 13248,
  'oil': 0,
  'unknown': 160,
  'hydro discharge': -129,
  'battery discharge': None},
 'powerImportBreakdown': {'BE': 107,
  'FR': 1624,
  'IE': 0,
  'IM': 0,
  'NL': 0,
  'GB-NIR': 0,
  'GB-ORK': 18,
  'NO-NO2': 1448},
 'powerExportBreakdown': {'BE': 0,
  'FR': 0,
  'IE': 530,
  'IM': 31,
  'NL': 424,
  'GB-NIR': 357,
  'GB-ORK': 0,
  'NO-NO2': 0},
 'fossilFreePercentage': 58,
 'renewablePercentage': 41,
 'powerConsumptionTotal': 31871,
 'powerPro

In [9]:
total_consumption = power_breakdown['powerConsumptionTotal']
total_consumption

31871

In [10]:
power_consumption_pct = {
    k:v/total_consumption*100
    for k,v
    in power_breakdown['powerConsumptionBreakdown'].items()
}

In [11]:
power_consumption_pct

{'nuclear': 16.830347337705124,
 'geothermal': 0.0,
 'biomass': 9.519625992281384,
 'coal': 0.0,
 'wind': 27.595619842490038,
 'solar': 0.0,
 'hydro': 3.677324213234602,
 'gas': 41.59894575005491,
 'oil': 0.006275297292209219,
 'unknown': 0.5679144049449343,
 'hydro discharge': 0.20708481064290418,
 'battery discharge': 0.0}

### Helper function

In [12]:
def get_latest_power_breakdown(coordinates: Dict[str, float]):
    assert 'lat' in coordinates
    assert 'lon' in coordinates
    
    url = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request = requests.get(url, headers={'auth-token': load_emaps_api_key()})
    power_breakdown = json.loads(request.content)
    
    total_consumption = power_breakdown['powerConsumptionTotal']
    power_consumption_pct = {
        k:v/total_consumption*100
        for k,v
        in power_breakdown['powerConsumptionBreakdown'].items()
    }
    return power_consumption_pct

In [13]:
get_latest_power_breakdown(coordinates)

{'nuclear': 16.830347337705124,
 'geothermal': 0.0,
 'biomass': 9.519625992281384,
 'coal': 0.0,
 'wind': 27.595619842490038,
 'solar': 0.0,
 'hydro': 3.677324213234602,
 'gas': 41.59894575005491,
 'oil': 0.006275297292209219,
 'unknown': 0.5679144049449343,
 'hydro discharge': 0.20708481064290418,
 'battery discharge': 0.0}

### Processing AWS datacenters

In [14]:
aws_regions = []

with open('aws_datacenters', 'r') as f:
    data = f.readlines()

for region in data:
    region = region.replace('\n', '').strip()
    parts = region.split(';')
    if len(parts) != 4:
        print('Unable to parse line', parts, 'Skipping...')
        continue
    aws_regions.append({
        'region': parts[0],
        'name': parts[1],
        'lat': parts[2],
        'lon': parts[3],
    })

Unable to parse line [''] Skipping...


In [15]:
aws_regions[:6]

[{'region': 'us-east-1', 'name': 'Virginia', 'lat': '38.13', 'lon': '-78.45'},
 {'region': 'us-east-2', 'name': 'Ohio', 'lat': '39.96', 'lon': '-83'},
 {'region': 'eu-north-1', 'name': 'Stockholm', 'lat': '59.25', 'lon': '17.81'},
 {'region': 'eu-south-1', 'name': 'Milan', 'lat': '45.43', 'lon': '9.29'},
 {'region': 'us-west-1',
  'name': 'California',
  'lat': '37.35',
  'lon': '-121.96'},
 {'region': 'us-west-2', 'name': 'Oregon', 'lat': '46.15', 'lon': '-123.88'}]

## Try coordinates of an AWS dataserver

In [16]:
print(aws_regions[0])
get_latest_power_breakdown(aws_regions[0])

{'region': 'us-east-1', 'name': 'Virginia', 'lat': '38.13', 'lon': '-78.45'}


{'nuclear': 33.115585064610556,
 'geothermal': 0.0,
 'biomass': 0.0,
 'coal': 10.406701411199162,
 'wind': 9.643987530044502,
 'solar': 0.001189881249851265,
 'hydro': 2.9782727683777157,
 'gas': 42.71435710716071,
 'oil': 0.5080792936864901,
 'unknown': 0.6318269436710217,
 'hydro discharge': 0.0,
 'battery discharge': 0.0}

In [17]:
print(aws_regions[2])
get_latest_power_breakdown(aws_regions[2])

{'region': 'eu-north-1', 'name': 'Stockholm', 'lat': '59.25', 'lon': '17.81'}


{'nuclear': 70.35670356703567,
 'geothermal': 0.0,
 'biomass': 0.0,
 'coal': 0.0,
 'wind': 2.127921279212792,
 'solar': 0.0,
 'hydro': 19.82779827798278,
 'gas': 0.0,
 'oil': 0.0,
 'unknown': 7.687576875768758,
 'hydro discharge': 0.0,
 'battery discharge': 0.0}

## Merge the regions and power breakdown datasets

In [18]:
data = []

In [19]:
for region in aws_regions:
    power_breakdown = get_latest_power_breakdown(region)
    
    # Determine highest contributing energy source
    highest_source = max(power_breakdown, key=power_breakdown.get)

    region.update(power_breakdown)
    region['highest_source'] = highest_source
    
    data.append(region)

In [20]:
data

[{'region': 'us-east-1',
  'name': 'Virginia',
  'lat': '38.13',
  'lon': '-78.45',
  'nuclear': 33.115585064610556,
  'geothermal': 0.0,
  'biomass': 0.0,
  'coal': 10.406701411199162,
  'wind': 9.643987530044502,
  'solar': 0.001189881249851265,
  'hydro': 2.9782727683777157,
  'gas': 42.71435710716071,
  'oil': 0.5080792936864901,
  'unknown': 0.6318269436710217,
  'hydro discharge': 0.0,
  'battery discharge': 0.0,
  'highest_source': 'gas'},
 {'region': 'us-east-2',
  'name': 'Ohio',
  'lat': '39.96',
  'lon': '-83',
  'nuclear': 33.115585064610556,
  'geothermal': 0.0,
  'biomass': 0.0,
  'coal': 10.406701411199162,
  'wind': 9.643987530044502,
  'solar': 0.001189881249851265,
  'hydro': 2.9782727683777157,
  'gas': 42.71435710716071,
  'oil': 0.5080792936864901,
  'unknown': 0.6318269436710217,
  'hydro discharge': 0.0,
  'battery discharge': 0.0,
  'highest_source': 'gas'},
 {'region': 'eu-north-1',
  'name': 'Stockholm',
  'lat': '59.25',
  'lon': '17.81',
  'nuclear': 70.3567

## Plot the data with Folium

In [21]:
import folium

In [22]:
map = folium.Map(tiles="OpenStreetMap", location=[31.8,0], zoom_start=2.2)

In [23]:
for region in data:
    highest_source = region['highest_source']
    if highest_source in ('gas', 'coal', 'oil', 'battery discharge'):
        colour = 'red'
    elif highest_source == 'nuclear':
        colour = 'orange'
    elif highest_source in ('hydro', 'biomass', 'solar', 'wind', 'geothermal', 'hydro discharge'):
        colour = 'green'
    else:
        colour = 'gray'

    energy_pct = round(float(region[highest_source]), 2)

    text = f'{region["region"]} ({region["name"]}): {highest_source} ({energy_pct}%)'
    
    marker = folium.Marker(
        location=(region['lat'], region['lon']),
        popup=text,
        icon=folium.Icon(color=colour)
    )
    
    map.add_child(marker)

map